In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn import linear_model

In [3]:
# Load the data
file_path = Path('Diabetes dataset_Group12.csv')
df = pd.read_csv(file_path)
df = df.drop(columns = "ID")
new_df = df.drop(columns = "Label")
new_df.head()

,FPG,HOMA_IR,Total_TAG,2hr_OGTT,T_CE,T_TAG,T_DAG,T_FFA,T_PC,T_PE,...,total DMA,alpha-AAA,Creatinine,Kynurenine,Met-SO,t4-OH-Pro,Putrescine,Serotonin,Spermidine,Taurine
0,6.426266,2.336573,5.781363,6.491854,12.09865,10.273816,4.086167,9.443437,10.80714,7.199930,...,0.111588,1.874979,6.710942,2.394366,-5.852678,4.214117,-3.462398,-1.018531,-1.642998,6.532927
1,6.741468,3.109857,7.906891,7.451211,12.09865,10.807138,4.907486,8.901539,10.27382,7.402848,...,-0.692381,-0.488355,6.008138,0.756260,-3.400239,2.677591,-3.313604,0.813043,-2.012510,7.612722
2,6.672426,3.621295,7.000001,6.942515,12.09865,10.273816,4.379796,8.802699,10.80714,6.759415,...,-0.032965,0.487615,6.008138,1.285267,-4.594876,3.496043,-7.375598,-1.504665,-2.644530,6.185453
3,6.491854,1.563326,7.011228,6.629358,12.09865,10.273816,4.281923,8.522484,10.80714,6.948541,...,0.226960,0.382100,6.440727,2.198089,-2.392424,4.504812,-3.638916,-2.520678,-2.473015,5.894638
4,6.539160,1.997820,6.285404,6.954197,12.09865,9.228706,3.965879,8.901539,10.80714,7.082903,...,1.018463,0.024295,6.532927,2.766317,-8.606643,3.550989,-2.483951,-1.491908,-1.194648,6.267509


In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Label'] = df['Label'].astype(str)
df['Label'] = encoder.fit_transform(df['Label'])

In [5]:
# Create our features
X = new_df

# Create our target
y = df["Label"]

In [6]:
X.describe()

,FPG,HOMA_IR,Total_TAG,2hr_OGTT,T_CE,T_TAG,T_DAG,T_FFA,T_PC,T_PE,...,total DMA,alpha-AAA,Creatinine,Kynurenine,Met-SO,t4-OH-Pro,Putrescine,Serotonin,Spermidine,Taurine
count,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,...,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000
mean,6.614549,2.701736,6.844076,6.926155,12.133278,10.319602,4.323889,8.893333,10.669212,7.054732,...,-0.254614,0.079763,6.135712,1.663323,-3.658187,3.805103,-3.972084,-2.809970,-2.167594,5.531168
std,0.139946,0.923355,0.822207,0.336438,0.128341,0.613956,0.329148,0.324070,0.235568,0.295159,...,0.840540,0.865511,0.491906,0.682331,2.969190,0.772432,1.907039,2.188667,0.738385,0.906282
min,6.339851,0.706960,5.584967,5.882646,12.098650,9.004653,3.531287,8.110500,10.273820,6.440727,...,-2.520678,-2.973741,4.784064,-0.014012,-10.044284,1.285267,-10.044283,-10.044283,-4.181955,3.590169
25%,6.515637,2.153218,6.228442,6.703892,12.098650,10.273816,4.090292,8.710862,10.407150,6.849232,...,-0.779078,-0.344862,5.824147,1.085826,-5.841102,3.483871,-4.613421,-3.881660,-2.587629,4.789947
50%,6.607312,2.538648,6.761414,6.906891,12.098650,10.273816,4.331543,8.901539,10.807140,7.043002,...,-0.145946,0.150635,6.244521,1.781592,-2.782335,3.868034,-3.405381,-2.407604,-2.127347,5.723925
75%,6.700441,3.319494,7.266787,7.199572,12.098650,10.673808,4.568362,9.111264,10.807140,7.243204,...,0.328442,0.473564,6.532927,2.162975,-1.191318,4.229380,-2.652177,-1.466755,-1.567777,6.185453
max,6.965785,5.093344,8.774787,7.507795,12.600750,11.408379,4.907486,9.443437,10.807140,7.769827,...,1.193675,2.141128,6.798392,3.029029,1.045463,5.202502,-1.147408,0.972990,-0.933018,7.612722


In [7]:
# Check the balance of our target values
y.value_counts()

1    30
0    28
Name: Label, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [9]:
# Resample the training data with the multiple linear regression model
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(kernel='linear')

In [10]:
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,0,0
1,0,1
2,1,1
3,0,1
4,0,1


In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6

In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.71      0.63         7
           1       0.67      0.50      0.57         8

    accuracy                           0.60        15
   macro avg       0.61      0.61      0.60        15
weighted avg       0.61      0.60      0.60        15



In [13]:
## Easy Ensemble AdaBoost Classifier

In [14]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easyensemb = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
easyensemb = easyensemb.fit(X_train, y_train)

In [16]:
# Calculated the balanced accuracy score
yprediction = easyensemb.predict(X_test)
balanced_accuracy_score(y_test, yprediction)

0.5982142857142857

In [19]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, yprediction)
print(matrix)

[[4 3]
 [3 5]]


In [20]:
# Print the imbalanced classification report

report = classification_report_imbalanced(y_test, yprediction)
print(report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.57      0.57      0.62      0.57      0.60      0.36         7
          1       0.62      0.62      0.57      0.62      0.60      0.36         8

avg / total       0.60      0.60      0.60      0.60      0.60      0.36        15



In [21]:
## oversampling

In [22]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 22, 1: 22})

In [23]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
naive = LogisticRegression(solver='lbfgs', random_state=1)
naive.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [37]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
yprediction = naive.predict(X_test)
balanced_accuracy_score(y_test, yprediction)

0.6785714285714286

In [38]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, yprediction)
print(matrix)

[[6 1]
 [4 4]]


In [39]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
report = classification_report_imbalanced(y_test, yprediction)
print(report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.60      0.86      0.50      0.71      0.65      0.44         7
          1       0.80      0.50      0.86      0.62      0.65      0.41         8

avg / total       0.71      0.67      0.69      0.66      0.65      0.43        15



In [40]:
## Undersampling

In [41]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 21, 1: 21})

In [42]:
# Train the Logistic Regression model using the resampled data
under = LogisticRegression(solver='lbfgs', random_state=1)
under.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [43]:
# Calculated the balanced accuracy score
yprediction = under.predict(X_test)
balanced_accuracy_score(y_test, yprediction)

0.6160714285714286

In [44]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, yprediction)
print(matrix)

[[6 1]
 [5 3]]


In [45]:
# Print the imbalanced classification report
report = classification_report_imbalanced(y_test, yprediction)
print(report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.55      0.86      0.38      0.67      0.57      0.34         7
          1       0.75      0.38      0.86      0.50      0.57      0.31         8

avg / total       0.65      0.60      0.63      0.58      0.57      0.32        15



In [46]:
##combination Sampling

In [47]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smoteenn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 10, 1: 1})

In [48]:
# Train the Logistic Regression model using the resampled data
comb = LogisticRegression(solver='lbfgs', random_state=1)
comb.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [49]:
# Calculated the balanced accuracy score
yprediction = comb.predict(X_test)
balanced_accuracy_score(y_test, yprediction)

0.6785714285714286

In [50]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, yprediction)
print(matrix)

[[6 1]
 [4 4]]


In [51]:
# Print the imbalanced classification report
report = classification_report_imbalanced(y_test, yprediction)
print(report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.60      0.86      0.50      0.71      0.65      0.44         7
          1       0.80      0.50      0.86      0.62      0.65      0.41         8

avg / total       0.71      0.67      0.69      0.66      0.65      0.43        15

